In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 第十章 数据聚合与分组运算

### Groupby机制
    分组键可以有多种形式，且类型不必相同:
    1. 列表或数组，其长度与待分组的轴一样。

    2. 表示DataFrame某个列名的值。

    3. 字典或Series，给出待分组轴上的值与分组名之间的对应关系。

    4. 函数，用于处理轴索引或索引中的各个标签。

In [22]:
df = pd.DataFrame({'key1': ['a','a','b','b','a'],
                  'key2': ['one', 'two','one','two','one'],
                  'data1': np.random.randn(5),
                  'data2': np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,-1.462803,0.883681
1,a,two,-0.808460,-0.004502
2,b,one,-0.513008,2.192351
3,b,two,-0.802503,-0.354411
4,a,one,-0.890532,0.523126


#### 以某列分组，计算另一列的平均值

In [3]:
grouped = df['data1'].groupby(df['key1'])      #按key1分组，计算data1列的平均值
grouped                                        #是一个Groupby对象。实际上未进行任何计算

In [4]:
grouped.mean()                                 #计算分组平均值

key1
a   -0.150200
b   -0.361019
Name: data1, dtype: float64

In [23]:
df[['data1']].groupby(df['key1'])               #此为 DataFrameGroupby object，聚合后是dataframe格式，
                                                #上述是Series格式

In [24]:
df[['data1']].groupby(df['key1']).mean()         #dataframe格式

,data1
key1,
a,-1.053932
b,-0.657756


In [5]:
# 一次传入多个数组
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     one    -0.635443
      two     0.820285
b     one    -0.813964
      two     0.091927
Name: data1, dtype: float64

In [6]:
means.unstack()

key2,one,two
key1,,
a,-0.635443,0.820285
b,-0.813964,0.091927


#### 1.分组键是Series。（分组键可以是任何长度适当的数组）

In [7]:
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])
df['data1'].groupby([states, years]).mean()

California  2005    0.820285
            2006   -0.813964
Ohio        2005   -0.681258
            2006    0.183558
Name: data1, dtype: float64

#### 2.分组键是列名

In [8]:
df.groupby('key1').mean()                  #没有key2列，因为key2不是数值列，所以从结果排除

,data1,data2
key1,,
a,-0.150200,0.724904
b,-0.361019,-0.342083


In [9]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one  -0.635443  0.745300
     two   0.820285  0.684112
b    one  -0.813964 -0.347631
     two   0.091927 -0.336535

#### Groupby的size方法，可以返回一个含有分组大小的Series

In [10]:
df.groupby(['key1', 'key2']).size()                    # 任何分组关键词中的缺失值，都会从结果中除去

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

In [11]:
df

,key1,key2,data1,data2
0,a,one,-1.454444,1.738105
1,a,two,0.820285,0.684112
2,b,one,-0.813964,-0.347631
3,b,two,0.091927,-0.336535
4,a,one,0.183558,-0.247505


In [33]:
df['data3'] = np.array([1,2,3,np.nan,5])                                    #添加一列含有null的值

In [36]:
#df[['data4']] = np.arange(5)                                                # 提示错误

In [37]:
df

,key1,key2,data1,data2,data3
0,a,one,-1.462803,0.883681,1.0
1,a,two,-0.808460,-0.004502,2.0
2,b,one,-0.513008,2.192351,3.0
3,b,two,-0.802503,-0.354411,NaN
4,a,one,-0.890532,0.523126,5.0


In [38]:
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

In [39]:
df1 = pd.Series(['a',np.nan,0.5,-0.1,6.0])

In [40]:
df2 = pd.concat([df, df1], axis=1)

In [41]:
df2                                                     #按axis=1进行拼接

,key1,key2,data1,data2,data3,0
0,a,one,-1.462803,0.883681,1.0,a
1,a,two,-0.808460,-0.004502,2.0,NaN
2,b,one,-0.513008,2.192351,3.0,0.5
3,b,two,-0.802503,-0.354411,NaN,-0.1
4,a,one,-0.890532,0.523126,5.0,6


In [42]:
df

,key1,key2,data1,data2,data3
0,a,one,-1.462803,0.883681,1.0
1,a,two,-0.808460,-0.004502,2.0
2,b,one,-0.513008,2.192351,3.0
3,b,two,-0.802503,-0.354411,NaN
4,a,one,-0.890532,0.523126,5.0


In [44]:
df2[0]

0      a
1    NaN
2    0.5
3   -0.1
4      6
Name: 0, dtype: object

In [45]:
#df.drop([0], axis=1, inplace=True)                                     #删除某列，原地删除 inplace=True
df.loc[5,:] = ['a',np.nan,0.5,-0.1,6.0]                                 #添加一行

In [46]:
df

,key1,key2,data1,data2,data3
0,a,one,-1.462803,0.883681,1.0
1,a,two,-0.808460,-0.004502,2.0
2,b,one,-0.513008,2.192351,3.0
3,b,two,-0.802503,-0.354411,NaN
4,a,one,-0.890532,0.523126,5.0
5,a,NaN,0.500000,-0.100000,6.0


In [47]:
df.groupby(['key1','key2']).mean()                       # 分组关键词中的空值被除去

data1     data2  data3
key1 key2                           
a    one  -1.176667  0.703404    3.0
     two  -0.808460 -0.004502    2.0
b    one  -0.513008  2.192351    3.0
     two  -0.802503 -0.354411    NaN

In [48]:
df.loc[6,:] = ['b', 'two', np.nan, np.nan, 7.0]           # 又添加一行带null的数据

In [49]:
df

,key1,key2,data1,data2,data3
0,a,one,-1.462803,0.883681,1.0
1,a,two,-0.808460,-0.004502,2.0
2,b,one,-0.513008,2.192351,3.0
3,b,two,-0.802503,-0.354411,NaN
4,a,one,-0.890532,0.523126,5.0
5,a,NaN,0.500000,-0.100000,6.0
6,b,two,NaN,NaN,7.0


In [50]:
df.groupby(['key1','key2']).mean()                       #  再次看分组求均值的变化，mean(null值+7.0) = 7.0

data1     data2  data3
key1 key2                           
a    one  -1.176667  0.703404    3.0
     two  -0.808460 -0.004502    2.0
b    one  -0.513008  2.192351    3.0
     two  -0.802503 -0.354411    7.0

In [51]:
df.groupby(['key1', 'key2']).size()                       #   此时再看计数，b two 应该变为2

key1  key2
a     one     2
      two     1
b     one     1
      two     2
dtype: int64

#### 对分组进行迭代

In [52]:
for name, group in df.groupby('key1'):
    print(name)

a
b


In [53]:
for name, group in df.groupby('key1'):                          #可以产生一组二元元组(由分组名和数据块组成)
    print(name)
    print(group)

a
  key1 key2     data1     data2  data3
0    a  one -1.462803  0.883681    1.0
1    a  two -0.808460 -0.004502    2.0
4    a  one -0.890532  0.523126    5.0
5    a  NaN  0.500000 -0.100000    6.0
b
  key1 key2     data1     data2  data3
2    b  one -0.513008  2.192351    3.0
3    b  two -0.802503 -0.354411    NaN
6    b  two       NaN       NaN    7.0


In [58]:
df

,key1,key2,data1,data2,data3
0,a,one,-1.462803,0.883681,1.0
1,a,two,-0.808460,-0.004502,2.0
2,b,one,-0.513008,2.192351,3.0
3,b,two,-0.802503,-0.354411,NaN
4,a,one,-0.890532,0.523126,5.0
5,a,NaN,0.500000,-0.100000,6.0
6,b,two,NaN,NaN,7.0


In [54]:
list(df.groupby('key1'))

[('a',   key1 key2     data1     data2  data3
  0    a  one -1.462803  0.883681    1.0
  1    a  two -0.808460 -0.004502    2.0
  4    a  one -0.890532  0.523126    5.0
  5    a  NaN  0.500000 -0.100000    6.0),
 ('b',   key1 key2     data1     data2  data3
  2    b  one -0.513008  2.192351    3.0
  3    b  two -0.802503 -0.354411    NaN
  6    b  two       NaN       NaN    7.0)]

In [59]:
list(df.groupby('key1')['data1'])

[('a', 0   -1.462803
  1   -0.808460
  4   -0.890532
  5    0.500000
  Name: data1, dtype: float64), ('b', 2   -0.513008
  3   -0.802503
  6         NaN
  Name: data1, dtype: float64)]

In [67]:
list(df['data1'].groupby(df['key1']))                       #上一行是这句的语法糖

[('a', 0   -1.462803
  1   -0.808460
  4   -0.890532
  5    0.500000
  Name: data1, dtype: float64), ('b', 2   -0.513008
  3   -0.802503
  6         NaN
  Name: data1, dtype: float64)]

In [57]:
list(df.groupby('key1')[['data1']])                     #看来这样的写法和下一行不等价

[('a',   key1 key2     data1     data2  data3
  0    a  one -1.462803  0.883681    1.0
  1    a  two -0.808460 -0.004502    2.0
  4    a  one -0.890532  0.523126    5.0
  5    a  NaN  0.500000 -0.100000    6.0),
 ('b',   key1 key2     data1     data2  data3
  2    b  one -0.513008  2.192351    3.0
  3    b  two -0.802503 -0.354411    NaN
  6    b  two       NaN       NaN    7.0)]

In [77]:
list(df.groupby('key1')[['data2']])

[('a',   key1 key2     data1     data2  data3
  0    a  one -1.462803  0.883681    1.0
  1    a  two -0.808460 -0.004502    2.0
  4    a  one -0.890532  0.523126    5.0
  5    a  NaN  0.500000 -0.100000    6.0),
 ('b',   key1 key2     data1     data2  data3
  2    b  one -0.513008  2.192351    3.0
  3    b  two -0.802503 -0.354411    NaN
  6    b  two       NaN       NaN    7.0)]

In [79]:
list(df[['data1']].groupby(df['key1']))                     #和上一行式子不等价，所以还是语法糖吗？

[('a',       data1
  0 -1.462803
  1 -0.808460
  4 -0.890532
  5  0.500000), ('b',       data1
  2 -0.513008
  3 -0.802503
  6       NaN)]

In [97]:
df[['data1']].groupby(df['key1']).mean()             #这两句的聚合运算结果是一致的

,data1
key1,
a,-0.665449
b,-0.657756


In [98]:
df.groupby(df['key1'])[['data1']].mean()             #这两句的聚合运算结果是一致的

,data1
key1,
a,-0.665449
b,-0.657756


# 这个问题思考了很久，暂时定为书本的意思是语法糖进行聚合运算等价，意即
# df[['data1']].groupby(df['key1']).mean() == df.groupby(df['key1'])[['data1']].mean()
# 都是一列DataFrame结构且相等，但是迭代或者list一下df.groupby(df['key1'])[['data1']]是全表而不是一列，这一点二者不一致。
# 下面两句迭代或者list都一致
# df.groupby('key1')['data1'] == df['data1'].groupby(df['key1'])
# df.groupby('key1')['data2'] == df['data2'].groupby(df['key1'])      这两句前者都是后者的语法糖
# df[['data1']].groupby(df['key1']) == DataFrameGroupBy ,也是data1列

# 以上详情见上例

In [80]:
pieces = dict(list(df.groupby('key1')))

In [82]:
pieces['b']

,key1,key2,data1,data2,data3
2,b,one,-0.513008,2.192351,3.0
3,b,two,-0.802503,-0.354411,NaN
6,b,two,NaN,NaN,7.0


In [83]:
pieces

{'a':   key1 key2     data1     data2  data3
 0    a  one -1.462803  0.883681    1.0
 1    a  two -0.808460 -0.004502    2.0
 4    a  one -0.890532  0.523126    5.0
 5    a  NaN  0.500000 -0.100000    6.0,
 'b':   key1 key2     data1     data2  data3
 2    b  one -0.513008  2.192351    3.0
 3    b  two -0.802503 -0.354411    NaN
 6    b  two       NaN       NaN    7.0}

In [84]:
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
data3    float64
dtype: object

#### 按数据类型分组

In [85]:
grouped = df.groupby(df.dtypes, axis=1)               #groupby默认是在axis=0上进行分组

In [86]:
for dtype, group in grouped:
    print(dtype)
    print(group)                                      #对分组进行迭代打印

float64
      data1     data2  data3
0 -1.462803  0.883681    1.0
1 -0.808460 -0.004502    2.0
2 -0.513008  2.192351    3.0
3 -0.802503 -0.354411    NaN
4 -0.890532  0.523126    5.0
5  0.500000 -0.100000    6.0
6       NaN       NaN    7.0
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one
5    a  NaN
6    b  two


In [87]:
df

,key1,key2,data1,data2,data3
0,a,one,-1.462803,0.883681,1.0
1,a,two,-0.808460,-0.004502,2.0
2,b,one,-0.513008,2.192351,3.0
3,b,two,-0.802503,-0.354411,NaN
4,a,one,-0.890532,0.523126,5.0
5,a,NaN,0.500000,-0.100000,6.0
6,b,two,NaN,NaN,7.0


In [93]:
df.groupby('key1')['data1']

In [94]:
df.groupby('key1')[['data1']]

In [99]:
for name, group in df.groupby('key1')['data1']:
    print(name)
    print(group)                                          #选取单个列

a
0   -1.462803
1   -0.808460
4   -0.890532
5    0.500000
Name: data1, dtype: float64
b
2   -0.513008
3   -0.802503
6         NaN
Name: data1, dtype: float64


In [100]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)                                          #选取整个表格

a
  key1 key2     data1     data2  data3
0    a  one -1.462803  0.883681    1.0
1    a  two -0.808460 -0.004502    2.0
4    a  one -0.890532  0.523126    5.0
5    a  NaN  0.500000 -0.100000    6.0
b
  key1 key2     data1     data2  data3
2    b  one -0.513008  2.192351    3.0
3    b  two -0.802503 -0.354411    NaN
6    b  two       NaN       NaN    7.0


In [101]:
for name, group in df.groupby('key1')[['data2']]:
    print(name)
    print(group)        

a
  key1 key2     data1     data2  data3
0    a  one -1.462803  0.883681    1.0
1    a  two -0.808460 -0.004502    2.0
4    a  one -0.890532  0.523126    5.0
5    a  NaN  0.500000 -0.100000    6.0
b
  key1 key2     data1     data2  data3
2    b  one -0.513008  2.192351    3.0
3    b  two -0.802503 -0.354411    NaN
6    b  two       NaN       NaN    7.0


In [102]:
df

,key1,key2,data1,data2,data3
0,a,one,-1.462803,0.883681,1.0
1,a,two,-0.808460,-0.004502,2.0
2,b,one,-0.513008,2.192351,3.0
3,b,two,-0.802503,-0.354411,NaN
4,a,one,-0.890532,0.523126,5.0
5,a,NaN,0.500000,-0.100000,6.0
6,b,two,NaN,NaN,7.0


In [103]:
df['data1']

0   -1.462803
1   -0.808460
2   -0.513008
3   -0.802503
4   -0.890532
5    0.500000
6         NaN
Name: data1, dtype: float64

In [105]:
type(df['data1'])

pandas.core.series.Series

In [104]:
df[['data1']]

,data1
0,-1.462803
1,-0.808460
2,-0.513008
3,-0.802503
4,-0.890532
5,0.500000
6,NaN


In [106]:
type(df[['data1']])

pandas.core.frame.DataFrame

In [107]:
df[['data1','data2']]

,data1,data2
0,-1.462803,0.883681
1,-0.808460,-0.004502
2,-0.513008,2.192351
3,-0.802503,-0.354411
4,-0.890532,0.523126
5,0.500000,-0.100000
6,NaN,NaN


In [108]:
df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    one   0.703404
     two  -0.004502
b    one   2.192351
     two  -0.354411

In [109]:
df.groupby(['key1', 'key2'])[['data2']]

In [110]:
df.groupby(['key1', 'key2'])['data2']

In [125]:
list(df.groupby(['key1', 'key2'])['data2'])

[(('a', 'one'), 0    0.883681
  4    0.523126
  Name: data2, dtype: float64), (('a', 'two'), 1   -0.004502
  Name: data2, dtype: float64), (('b', 'one'), 2    2.192351
  Name: data2, dtype: float64), (('b', 'two'), 3   -0.354411
  6         NaN
  Name: data2, dtype: float64)]

In [111]:
df.groupby(['key1', 'key2'])['data2'].mean()

key1  key2
a     one     0.703404
      two    -0.004502
b     one     2.192351
      two    -0.354411
Name: data2, dtype: float64

In [112]:
df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    one   0.703404
     two  -0.004502
b    one   2.192351
     two  -0.354411

In [133]:
#df[['data2']].groupby(df['key1','key2'])                 #会报错，

In [127]:
#for name, group in df[['data2']].groupby(df['key1'],df['key2']):
#    print(name)
#    print(group)                                                #会报错，这样的格式不行

In [132]:
#for name, group in df['data2'].groupby(df['key1','key2']):
#    print(name)
#    print(group)                                                #会报错，正确格式如下

In [126]:
for name, group in df.groupby(['key1', 'key2'])['data2']:
    print(name)
    print(group)

('a', 'one')
0    0.883681
4    0.523126
Name: data2, dtype: float64
('a', 'two')
1   -0.004502
Name: data2, dtype: float64
('b', 'one')
2    2.192351
Name: data2, dtype: float64
('b', 'two')
3   -0.354411
6         NaN
Name: data2, dtype: float64


In [139]:
for name,group in df.groupby(['key1','key2'])[['data2']]:
    print(name)
    print(group)                                                     #df.groupby [[]] 仍是全部

('a', 'one')
  key1 key2     data1     data2  data3
0    a  one -1.462803  0.883681    1.0
4    a  one -0.890532  0.523126    5.0
('a', 'two')
  key1 key2    data1     data2  data3
1    a  two -0.80846 -0.004502    2.0
('b', 'one')
  key1 key2     data1     data2  data3
2    b  one -0.513008  2.192351    3.0
('b', 'two')
  key1 key2     data1     data2  data3
3    b  two -0.802503 -0.354411    NaN
6    b  two       NaN       NaN    7.0


In [140]:
for name,group in df.groupby(['key1','key2']):
    print(name)
    print(group)                                                    #结果和上述相同

('a', 'one')
  key1 key2     data1     data2  data3
0    a  one -1.462803  0.883681    1.0
4    a  one -0.890532  0.523126    5.0
('a', 'two')
  key1 key2    data1     data2  data3
1    a  two -0.80846 -0.004502    2.0
('b', 'one')
  key1 key2     data1     data2  data3
2    b  one -0.513008  2.192351    3.0
('b', 'two')
  key1 key2     data1     data2  data3
3    b  two -0.802503 -0.354411    NaN
6    b  two       NaN       NaN    7.0


In [141]:
for name,group in df.groupby(df['key1']):
    print(name)
    print(group)

a
  key1 key2     data1     data2  data3
0    a  one -1.462803  0.883681    1.0
1    a  two -0.808460 -0.004502    2.0
4    a  one -0.890532  0.523126    5.0
5    a  NaN  0.500000 -0.100000    6.0
b
  key1 key2     data1     data2  data3
2    b  one -0.513008  2.192351    3.0
3    b  two -0.802503 -0.354411    NaN
6    b  two       NaN       NaN    7.0


#### 分组信息还可以有其他形式，如字典，Series

In [143]:
people = pd.DataFrame(np.random.randn(5,5),
                     columns=['a','b','c','d','e'],
                     index=['Joe','Steve','Wes','Jim','Travis'])
people

,a,b,c,d,e
Joe,0.641611,-1.387059,0.269170,2.037908,-1.060853
Steve,0.208898,-0.011628,0.855887,0.115191,0.097580
Wes,-0.259493,-0.716924,-0.855932,0.177552,-0.055560
Jim,-1.017261,-0.617057,-0.473341,1.382048,0.024687
Travis,-1.582270,0.511792,0.975813,-2.359172,-1.556189


In [144]:
people.iloc[2:3, [1,2]] = np.nan         # Add a few NA values
people

,a,b,c,d,e
Joe,0.641611,-1.387059,0.269170,2.037908,-1.060853
Steve,0.208898,-0.011628,0.855887,0.115191,0.097580
Wes,-0.259493,NaN,NaN,0.177552,-0.055560
Jim,-1.017261,-0.617057,-0.473341,1.382048,0.024687
Travis,-1.582270,0.511792,0.975813,-2.359172,-1.556189


In [145]:
mapping = {'a':'red', 'b':'red', 'c':'blue', 'd':'blue', 'e':'red', 'f':'orange'}

by_column = people.groupby(mapping, axis=1)                          # 可以传递字典，存在未使用的键也可以
by_column

In [146]:
by_column.sum()

,blue,red
Joe,2.307078,-1.806300
Steve,0.971078,0.294850
Wes,0.177552,-0.315054
Jim,0.908707,-1.609632
Travis,-1.383358,-2.626668


In [147]:
map_series = pd.Series(mapping)                         #  将字典转为 Series格式
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [148]:
map_dataframe = pd.DataFrame(map_series,columns=['x'])   #   将Series格式转为dataframe格式

In [149]:
map_dataframe

,x
a,red
b,red
c,blue
d,blue
e,red
f,orange


In [150]:
map_dataframe = pd.DataFrame(mapping,index=['x'])        #    将字典转为dataframe格式，(字典键代表columns)

In [151]:
map_dataframe

,a,b,c,d,e,f
x,red,red,blue,blue,red,orange


In [152]:
people.groupby(map_series, axis=1).size()

blue    2
red     3
dtype: int64

In [153]:
people.groupby(map_series, axis=1).count()                  #根据分组计算列的和

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


#### 通过函数进行分组

In [154]:
people

,a,b,c,d,e
Joe,0.641611,-1.387059,0.269170,2.037908,-1.060853
Steve,0.208898,-0.011628,0.855887,0.115191,0.097580
Wes,-0.259493,NaN,NaN,0.177552,-0.055560
Jim,-1.017261,-0.617057,-0.473341,1.382048,0.024687
Travis,-1.582270,0.511792,0.975813,-2.359172,-1.556189


In [155]:
people.groupby(len).sum()          #任何可以被当做分组键的函数都会在各个索引值上被调用一次，
                                   #其返回值被用作分组名称

,a,b,c,d,e
3,-0.635143,-2.004116,-0.204171,3.597509,-1.091727
5,0.208898,-0.011628,0.855887,0.115191,0.097580
6,-1.582270,0.511792,0.975813,-2.359172,-1.556189


#### 将函数跟数组、列表、字典、Series混合使用，因为任何东西在内部都会转换为数组

In [156]:
key_list = ['one', 'one', 'one', 'two', 'two']
people.groupby([len, key_list]).min()

a         b         c         d         e
3 one -0.259493 -1.387059  0.269170  0.177552 -1.060853
  two -1.017261 -0.617057 -0.473341  1.382048  0.024687
5 one  0.208898 -0.011628  0.855887  0.115191  0.097580
6 two -1.582270  0.511792  0.975813 -2.359172 -1.556189

#### 根据索引级别分组

In [157]:
columns = pd.MultiIndex.from_arrays([['US','US','US','JP','JP'],
                                    [1,3,5,1,3]],
                                   names=['cty', 'tenor'])
hie_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)           #层次化索引

In [158]:
columns

MultiIndex(levels=[['JP', 'US'], [1, 3, 5]],
           labels=[[1, 1, 1, 0, 0], [0, 1, 2, 0, 1]],
           names=['cty', 'tenor'])

In [159]:
hie_df

cty          US                            JP          
tenor         1         3         5         1         3
0     -0.224902 -0.513145  0.643900  1.265597 -1.050073
1     -1.173570 -0.457740  0.748011  0.442830 -0.091790
2      1.120185  1.442908 -0.097429 -0.341307  0.824177
3      0.267085  0.178007  0.848530  1.997440 -0.457088

In [160]:
hie = pd.DataFrame(np.random.randn(5,5), index=columns)       #自创:  假如是双重索引

In [161]:
hie

0         1         2         3         4
cty tenor                                                  
US  1     -0.194764 -2.037661 -1.643589  1.028631 -0.633350
    3     -0.795026 -2.106622  1.948829  1.863870 -0.186847
    5      0.975338  0.825989 -0.799466 -1.430403  0.936126
JP  1      0.225979 -0.622841 -0.577676 -0.262036  2.664045
    3      0.678737  0.082429 -0.122133 -0.170305  0.181294

In [162]:
hie_df.groupby(level='cty', axis=1).count()                #用level关键字传递级别序号或名字

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


## 数据聚合

##### quantile是Series方法，计算Series或DataFrame列的样本分位数

In [163]:
df

,key1,key2,data1,data2,data3
0,a,one,-1.462803,0.883681,1.0
1,a,two,-0.808460,-0.004502,2.0
2,b,one,-0.513008,2.192351,3.0
3,b,two,-0.802503,-0.354411,NaN
4,a,one,-0.890532,0.523126,5.0
5,a,NaN,0.500000,-0.100000,6.0
6,b,two,NaN,NaN,7.0


In [181]:
list(df.groupby('key1')['data1'])

[('a', 0   -1.462803
  1   -0.808460
  4   -0.890532
  5    0.500000
  Name: data1, dtype: float64), ('b', 2   -0.513008
  3   -0.802503
  6         NaN
  Name: data1, dtype: float64)]

In [185]:
df.groupby('key1')['data1'].quantile(0)               #计算分位数

key1
a   -1.462803
b   -0.802503
Name: data1, dtype: float64

In [186]:
df.groupby('key1')['data1'].quantile(0.5)             #计算中位数

key1
a   -0.849496
b   -0.657756
Name: data1, dtype: float64

In [187]:
df.groupby('key1')['data1'].quantile(0.9)             #计算分位数

key1
a    0.107462
b   -0.541957
Name: data1, dtype: float64

# 计算分位数
###    首先将你的输入数组进行从小到大排序，然后计算:
###    $(n−1)∗p=i+j(n−1)∗p=i+j$
###    其中n为数组元素的个数，将计算结果的整数部分用i表示，小数部分用j来表
###    示。则最终的percentile值为：
###    $res=(1−j)∗array[i]+j∗array[i+1]$
--------------------- 
作者：BugFree_张瑞 
来源：CSDN 
原文：https://blog.csdn.net/u011489043/article/details/78056851 
版权声明：本文为博主原创文章，转载请附上博文链接！

In [166]:
df.groupby('key1')['data1'].sum()

key1
a   -2.661795
b   -1.315511
Name: data1, dtype: float64

In [168]:
df.groupby('key1')['data1'].mean()

key1
a   -0.665449
b   -0.657756
Name: data1, dtype: float64

In [172]:
df.groupby('key1')['data1'].quantile(0.9)     

key1
a    0.107462
b   -0.541957
Name: data1, dtype: float64

In [173]:
a = np.array(([1,2,3,4]))
np.median(a)

2.5

#### 数据聚合第二点: 传入aggregate或agg方法,使用自己的聚合函数

In [195]:
def peak_to_peak(arr):
    return arr.max() - arr.min()
df.groupby('key1')['data1'].agg(peak_to_peak)

key1
a    1.962803
b    0.289495
Name: data1, dtype: float64

In [196]:
# 有些方法也可以使用，但严格讲不是聚合运算
df.groupby('key1')['data1'].describe()

,count,mean,std,min,25%,50%,75%,max
key1,,,,,,,,
a,4.0,-0.665449,0.829691,-1.462803,-1.033600,-0.849496,-0.481345,0.500000
b,2.0,-0.657756,0.204704,-0.802503,-0.730129,-0.657756,-0.585382,-0.513008


In [197]:
df.groupby('key1').describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      4.0 -0.665449  0.829691 -1.462803 -1.033600 -0.849496 -0.481345   
b      2.0 -0.657756  0.204704 -0.802503 -0.730129 -0.657756 -0.585382   

               data2           ...                      data3                 \
           max count      mean ...        75%       max count mean       std   
key1                           ...                                             
a     0.500000   4.0  0.325576 ...   0.613265  0.883681   4.0  3.5  2.380476   
b    -0.513008   2.0  0.918970 ...   1.555661  2.192351   2.0  5.0  2.828427   

                                 
      min   25%  50%   75%  max  
key1                             
a     1.0  1.75  3.5  5.25  6.0  
b     3.0  4.00  5.0  6.00  7.0  

[2 rows x 24 columns]

##### 自定义聚合函数要比经过优化的函数(max,min)慢得多，因为会有很大开销(函数调用、数据重排)

#### 面向列的多函数应用，在这里用的‘小费’数据集
#### 1. tips.groupby(['day', 'smoker'])['tip_pct'].agg(['mean', 'std', peak_to_peak]),
#### 2. ftuples = [('DUR', 'mean'), ('Abwe', np.var)]   grouped['tip_pct'].agg(ftuples)
#### 3. grouped.agg({'tip' : np.max, 'size' : 'sum'})
#### 4. grouped.agg({'tip_pct' : ['min', 'max', 'mean', 'std'], 'size' : 'sum'})
### 只有将多个函数应用到至少一列时(tips.groupby(xx)['vv']),
### dataframe才有层次化的列

In [198]:
df = pd.DataFrame(data={'book': ['bk1', 'bk1', 'bk1', 'bk2', 'bk2', 'bk3'], 
                        'price': [12, 12, 12, 15, 15, 17]})
df

,book,price
0,bk1,12
1,bk1,12
2,bk1,12
3,bk2,15
4,bk2,15
5,bk3,17


In [200]:
df.groupby('book').sum()                         #默认ax_index=True

,price
book,
bk1,36
bk2,30
bk3,17


In [201]:
df.groupby('book', as_index=False).sum()         #取消分组键作索引

,book,price
0,bk1,36
1,bk2,30
2,bk3,17


# 小费数据例子

In [203]:
frame = pd.DataFrame({'data1': np.random.randn(1000),
                     'data2': np.random.randn(1000)})
quartiles = pd.cut(frame.data1, 4)
quartiles[:10]

0    (-2.087, -0.552]
1     (-0.552, 0.982]
2     (-0.552, 0.982]
3    (-2.087, -0.552]
4     (-0.552, 0.982]
5     (-0.552, 0.982]
6     (-0.552, 0.982]
7    (-2.087, -0.552]
8     (-0.552, 0.982]
9    (-2.087, -0.552]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-3.627, -2.087] < (-2.087, -0.552] < (-0.552, 0.982] < (0.982, 2.516]]

In [206]:
grouped = frame.data2.groupby(quartiles)

In [209]:
# 由cut返回的Categorical对象可直接传递到groupby
def get_stats(group):
    return {'min': group.min(), 'max': group.max(),
           'count': group.count(), 'mean': group.mean()}

In [210]:
list(grouped)

[(Interval(-3.627, -2.087, closed='right'), 31     2.747524
  36    -0.126037
  93    -0.292046
  103    0.406231
  137   -0.451166
  208    0.660413
  290    0.016804
  465    1.815242
  485   -0.607565
  621    1.481082
  626   -1.607892
  720   -2.067778
  834    0.366883
  872    0.594811
  881    0.887474
  944   -1.372547
  Name: data2, dtype: float64),
 (Interval(-2.087, -0.552, closed='right'), 0      0.652247
  3      1.370519
  7     -1.284391
  9     -0.552780
  11    -0.660968
  16    -0.794238
  17    -1.255947
  18     1.409306
  23    -0.922795
  24    -0.961754
  26     0.726474
  28     0.178320
  49    -1.145267
  62     0.329293
  65    -0.477811
  66     0.577077
  67    -1.233985
  69     1.346328
  70    -2.349843
  72    -0.703335
  79     1.185970
  81     0.282401
  82    -0.290190
  84    -1.100908
  90    -0.847045
  91    -1.701602
  92     0.135486
  94    -0.874493
  95    -0.849378
  99     0.478748
           ...   
  891    0.001166
  894   -0.821792
  

In [214]:
grouped.apply(get_stats)                             #apply运用

data1                  
(-3.627, -2.087]  count     16.000000
                  max        2.747524
                  mean       0.153215
                  min       -2.067778
(-2.087, -0.552]  count    270.000000
                  max        3.130865
                  mean       0.004477
                  min       -3.463098
(-0.552, 0.982]   count    567.000000
                  max        2.742843
                  mean       0.037466
                  min       -2.514495
(0.982, 2.516]    count    147.000000
                  max        2.505708
                  mean       0.180957
                  min       -1.770850
Name: data2, dtype: float64

In [215]:
grouped.apply(get_stats).unstack()

,count,max,mean,min
data1,,,,
"(-3.627, -2.087]",16.0,2.747524,0.153215,-2.067778
"(-2.087, -0.552]",270.0,3.130865,0.004477,-3.463098
"(-0.552, 0.982]",567.0,2.742843,0.037466,-2.514495
"(0.982, 2.516]",147.0,2.505708,0.180957,-1.770850


In [219]:
# 根据样本分位数得到大小相等的桶，使用qcut即可。
grouping = pd.qcut(frame.data1, 10, labels=False)             # 传入labels=False即可只获取分位数的编号
grouping

0      0
1      3
2      6
3      0
4      7
5      3
6      3
7      1
8      3
9      0
10     8
11     0
12     4
13     8
14     9
15     9
16     2
17     1
18     1
19     8
20     3
21     6
22     3
23     0
24     2
25     3
26     0
27     7
28     2
29     3
      ..
970    0
971    0
972    9
973    5
974    8
975    8
976    3
977    8
978    6
979    3
980    0
981    7
982    1
983    5
984    6
985    3
986    7
987    4
988    7
989    7
990    5
991    2
992    7
993    6
994    0
995    5
996    0
997    7
998    8
999    2
Name: data1, Length: 1000, dtype: int64

In [220]:
grouped = frame.data2.groupby(grouping)
grouped

In [221]:
list(grouped)

[(0, 0      0.652247
  3      1.370519
  9     -0.552780
  11    -0.660968
  23    -0.922795
  26     0.726474
  31     2.747524
  36    -0.126037
  66     0.577077
  69     1.346328
  93    -0.292046
  99     0.478748
  102    0.386654
  103    0.406231
  127    0.954576
  137   -0.451166
  159   -1.698062
  179    0.441550
  184    1.008553
  198   -1.650440
  208    0.660413
  214   -0.058399
  224    0.502455
  232   -1.493356
  238   -0.253351
  248    2.019577
  268    1.511012
  271    1.336470
  290    0.016804
  293    1.799924
           ...   
  669   -1.227817
  675    0.691205
  679   -2.268502
  700   -1.597687
  702   -2.112383
  717    1.096869
  720   -2.067778
  732   -0.286720
  734   -0.739196
  741   -1.043711
  794   -1.252020
  805   -0.418524
  813    0.440646
  834    0.366883
  844    0.522089
  853    0.655675
  861    1.472587
  872    0.594811
  881    0.887474
  903   -0.276630
  938    1.829773
  944   -1.372547
  953   -1.829779
  954    0.868967
  965  

In [222]:
grouped.apply(get_stats)

data1       
0      count    100.000000
       max        2.747524
       mean       0.068079
       min       -3.463098
1      count    100.000000
       max        3.130865
       mean       0.014121
       min       -2.621084
2      count    100.000000
       max        2.238311
       mean      -0.072021
       min       -2.081337
3      count    100.000000
       max        2.575298
       mean       0.070515
       min       -2.272793
4      count    100.000000
       max        2.391982
       mean       0.033563
       min       -2.324176
5      count    100.000000
       max        2.742843
       mean       0.154418
       min       -1.816960
6      count    100.000000
       max        1.870834
       mean      -0.145033
       min       -2.479866
7      count    100.000000
       max        2.686151
       mean       0.069307
       min       -2.514495
8      count    100.000000
       max        2.684313
       mean       0.203223
       min       -2.345403
9      count   

In [223]:
grouped.apply(get_stats).unstack()

,count,max,mean,min
data1,,,,
0,100.0,2.747524,0.068079,-3.463098
1,100.0,3.130865,0.014121,-2.621084
2,100.0,2.238311,-0.072021,-2.081337
3,100.0,2.575298,0.070515,-2.272793
4,100.0,2.391982,0.033563,-2.324176
5,100.0,2.742843,0.154418,-1.816960
6,100.0,1.870834,-0.145033,-2.479866
7,100.0,2.686151,0.069307,-2.514495
8,100.0,2.684313,0.203223,-2.345403


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### 示例1: 用特定于分组的值填充缺失值
    对于缺失数据的清理工作，可能希望用一个固定值或由数据集本身衍生的值填充NA值

In [9]:
s = pd.Series(np.arange(6))

s[::2] = np.nan                              #索引[0],[2],[4]为空值
s

0    NaN
1    1.0
2    NaN
3    3.0
4    NaN
5    5.0
dtype: float64

In [10]:
s.fillna(s.mean())                           #用means填充缺失值, 已知数据求和 

0    3.0
1    1.0
2    3.0
3    3.0
4    3.0
5    5.0
dtype: float64

In [34]:
states = ['Ohio', 'New York', 'Vermont', 'Florida',
         'Oregon', 'Nevada', 'California', 'Idaho']

group_key = ['East'] * 4 + ['West'] * 4
data = pd.Series(np.random.randn(8), index = states)
data                                                          #Series格式数据

Ohio         -0.202725
New York     -1.281864
Vermont      -0.259180
Florida       0.208547
Oregon        0.098611
Nevada       -0.741699
California    0.601560
Idaho         1.071459
dtype: float64

In [26]:
data[2]

-1.1976023327208354

In [21]:
data.iloc[[2]]

Vermont   -1.197602
dtype: float64

In [22]:
data[[2]]

Vermont   -1.197602
dtype: float64

In [23]:
data['Vermont']

-1.1976023327208354

In [24]:
data[['Vermont']]

Vermont   -1.197602
dtype: float64

In [32]:
data.loc[['Vermont']]

Vermont   -1.197602
dtype: float64

In [33]:
data1 = np.array(np.random.randn(8))                   #ndarray格式 与 Series格式 索引异同

In [27]:
data1

array([ 1.67696972, -0.38461911,  1.63203281, -0.16261111,  0.17949147,
        0.67405343, -1.0631283 ,  0.94275769])

In [28]:
data1[2]

1.63203281364645

In [29]:
data1[[2]]

array([1.63203281])

In [31]:
# data1.iloc[2]  报错                                         pandas 的方法 iloc, loc, 

In [35]:
data

Ohio         -0.202725
New York     -1.281864
Vermont      -0.259180
Florida       0.208547
Oregon        0.098611
Nevada       -0.741699
California    0.601560
Idaho         1.071459
dtype: float64

In [36]:
data[['Vermont', 'Nevada', 'Idaho']] = np.nan
data

Ohio         -0.202725
New York     -1.281864
Vermont            NaN
Florida       0.208547
Oregon        0.098611
Nevada             NaN
California    0.601560
Idaho              NaN
dtype: float64

In [37]:
group_key

['East', 'East', 'East', 'East', 'West', 'West', 'West', 'West']

In [38]:
data.groupby(group_key).mean()                            # 传入一串数组 以其为分组键

East   -0.425347
West    0.350085
dtype: float64

In [39]:
fill_mean = lambda g: g.fillna(g.mean())
data.groupby(group_key).apply(fill_mean)                 # 用分组平均值取填充NA值

Ohio         -0.202725
New York     -1.281864
Vermont      -0.425347
Florida       0.208547
Oregon        0.098611
Nevada        0.350085
California    0.601560
Idaho         0.350085
dtype: float64

In [40]:
# 也可以在代码中预定义各组的填充值，分组有一个name属性
fill_values = {'East': 0.5, 'West' : -1}
fill_func = lambda g: g.fillna(fill_values[g.name])
data.groupby(group_key).apply(fill_func)

Ohio         -0.202725
New York     -1.281864
Vermont       0.500000
Florida       0.208547
Oregon        0.098611
Nevada       -1.000000
California    0.601560
Idaho        -1.000000
dtype: float64

### 示例2: 随机采样和排列

In [42]:
suits = ['H', 'S', 'C', 'D']
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ['A'] + list(range(2, 11)) + ['J', 'K', 'Q']

In [44]:
suits

['H', 'S', 'C', 'D']

In [45]:
card_val

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 10,
 10,
 10,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 10,
 10,
 10,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 10,
 10,
 10,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 10,
 10,
 10]

In [46]:
base_names

['A', 2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'K', 'Q']

In [48]:
cards = []
for suit in ['H', 'S', 'C', 'D']:
    cards.extend(str(num) + suit for num in base_names)
    
cards

['AH',
 '2H',
 '3H',
 '4H',
 '5H',
 '6H',
 '7H',
 '8H',
 '9H',
 '10H',
 'JH',
 'KH',
 'QH',
 'AS',
 '2S',
 '3S',
 '4S',
 '5S',
 '6S',
 '7S',
 '8S',
 '9S',
 '10S',
 'JS',
 'KS',
 'QS',
 'AC',
 '2C',
 '3C',
 '4C',
 '5C',
 '6C',
 '7C',
 '8C',
 '9C',
 '10C',
 'JC',
 'KC',
 'QC',
 'AD',
 '2D',
 '3D',
 '4D',
 '5D',
 '6D',
 '7D',
 '8D',
 '9D',
 '10D',
 'JD',
 'KD',
 'QD']

In [49]:
deck = pd.Series(card_val, index=cards)

In [50]:
deck[:13]

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

In [51]:
def draw(deck, n=5):                           #从整副牌中抽出5张
    return deck.sample(n)

draw(deck)

KH    10
7C     7
QS    10
8H     8
9H     9
dtype: int64

In [53]:
# 从每种花色中随机抽取两张牌

get_suit = lambda card: card[-1]           # last letter is suit
deck.groupby(get_suit).apply(draw, n=2)

C  5C     5
   JC    10
D  5D     5
   KD    10
H  2H     2
   KH    10
S  7S     7
   8S     8
dtype: int64

In [55]:
list(deck.groupby(get_suit))

[('C', AC      1
  2C      2
  3C      3
  4C      4
  5C      5
  6C      6
  7C      7
  8C      8
  9C      9
  10C    10
  JC     10
  KC     10
  QC     10
  dtype: int64), ('D', AD      1
  2D      2
  3D      3
  4D      4
  5D      5
  6D      6
  7D      7
  8D      8
  9D      9
  10D    10
  JD     10
  KD     10
  QD     10
  dtype: int64), ('H', AH      1
  2H      2
  3H      3
  4H      4
  5H      5
  6H      6
  7H      7
  8H      8
  9H      9
  10H    10
  JH     10
  KH     10
  QH     10
  dtype: int64), ('S', AS      1
  2S      2
  3S      3
  4S      4
  5S      5
  6S      6
  7S      7
  8S      8
  9S      9
  10S    10
  JS     10
  KS     10
  QS     10
  dtype: int64)]

In [56]:
# 也可以这样写
deck.groupby(get_suit, group_keys=False).apply(draw, n=2)

5C     5
9C     9
QD    10
9D     9
9H     9
5H     5
9S     9
5S     5
dtype: int64